In [4]:
# Enable all CPU cores
from tqdm import tqdm
import pandas as pd
import re
import pandas as pd

# Read the file content
def parse_to_datafame(file_path):
  with open(file_path, "r") as file:
      content = file.read()
  # Define a regex pattern to extract the test case and its details
  pattern = r"\s+structure\s+(?P<structure>.+)\s+nat\s+(?P<nat>\d+)\s+nprim\s+(?P<nprim>\d+)\s+maxl\s+(?P<maxl>\d+)\s+max_nnl\s+(?P<max_nnl>\d+)\s+mean_nnl\s+(?P<mean_nnl>\d+)\s+max_nsh\s+(?P<max_nsh>\d+)\s+total_nao\s+(?P<total_nao>\d+)\s+cpu_time\s+(?P<cpu_time>[\d.]+)"

  # Use re.finditer to extract all matches
  matches = re.finditer(pattern, content)

  # Create a list of dictionaries to store the extracted data
  data = []
  for match in matches:
      data.append(match.groupdict())

  # Convert the list of dictionaries into a pandas DataFrame
  df = pd.DataFrame(data)

# Convert numeric columns to appropriate data types
  numeric_columns = [
    'nat', 'nprim', 'maxl', 'max_nnl', 'mean_nnl', 'max_nsh', 'total_nao',
    'cpu_time'
  ]
  # Convert the numeric columns to float
  df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

  # Display the DataFrame
  df = df.sort_values('nat')
  return df

option = 'i9-14900HX-no-atomic'
!mkdir -p notes/capture/{option}
!mkdir -p notes/data/{option}

In [5]:
for ncore in tqdm([1, 2, 4, 8, 16, 32]):
  ncore = str(ncore)
  !NO_TEST_GPU=1 OMP_NUM_THREADS={ncore} meson test -C build/ hamiltonian --verbose -t 0 > notes/capture/{option}/{ncore}cpu.txt
  df = parse_to_datafame(f"notes/capture/{option}/{ncore}cpu.txt")
  df.to_csv(f"notes/data/{option}/{ncore}cpu.csv", index=False)

100%|██████████| 6/6 [00:14<00:00,  2.48s/it]


In [6]:
!NO_TEST_GPU=1 meson test -C build/ hamiltonian --verbose -t 0 > notes/capture/{option}/32allcpu.txt
df = parse_to_datafame(f"notes/capture/{option}/32allcpu.txt")
df.to_csv(f"notes/data/{option}/32allcpu.csv", index=False)

In [3]:
!NO_TEST_GPU=1 meson test -C build/ hamiltonian-batch --verbose -t 0 > notes/capture/{option}/32batchallcpu.txt

^C


In [7]:
!OMP_NUM_THREADS=1 NO_TEST_GPU=1 meson test -C build/ hamiltonian-batch --verbose -t 0 > notes/capture/{option}/1batchcpu.txt

^C


In [ ]:

def parse_to_datafame(file_path):
  with open(file_path, "r") as file:
      content = file.read()
  # Define a regex pattern to extract the test case and its details
  pattern = r"\s+cpu_time\s+(?P<cpu_time>[\d.]+)\s+batch_size\s+(?P<batch_size>[\d.]+)\s"

  # Use re.finditer to extract all matches
  matches = re.finditer(pattern, content)

  # Create a list of dictionaries to store the extracted data
  data = []
  for match in matches:
      data.append(match.groupdict())

  # Convert the list of dictionaries into a pandas DataFrame
  df = pd.DataFrame(data)

# Convert numeric columns to appropriate data types
  numeric_columns = [
    'cpu_time', 'batch_size'
  ]
  # Convert the numeric columns to float
  df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

  # Display the DataFrame
  df = df.sort_values('batch_size')
  return df


df = parse_to_datafame(f"notes/capture/{option}/32batchallcpu.txt")
df.to_csv(f"notes/data/{option}/32batchallcpu.csv", index=False)
df

,cpu_time,batch_size
0,23.146714,1
1,41.541485,4
2,25.911044,16
3,54.858065,64
4,226.617542,256
5,928.558061,1024
6,3380.195741,4096
7,13606.760627,16384


In [8]:
df = parse_to_datafame(f"notes/capture/{option}/1batchcpu.txt")
df.to_csv(f"notes/data/{option}/1batchcpu.csv", index=False)
df

,cpu_time,batch_size
0,13.515938,1
1,41.075046,4
2,167.830571,16
3,663.315850,64
4,2575.419801,256


In [3]:
!nproc
!lscpu

32
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          39 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   32
  On-line CPU(s) list:    0-31
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Core(TM) i9-14900HX
    CPU family:           6
    Model:                183
    Thread(s) per core:   2
    Core(s) per socket:   24
    Socket(s):            1
    Stepping:             1
    CPU(s) scaling MHz:   31%
    CPU max MHz:          5800.0000
    CPU min MHz:          800.0000
    BogoMIPS:             4838.40
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 s
                          s ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc 
                          art arch_perfmon pebs bts rep_good nopl xtopology nons
                          top_tsc cpuid